In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
import os
import gzip

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
source_dir = "./generate_csv/"
print(os.listdir(source_dir))

def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results

train_filenames = get_filenames_by_prefix(source_dir, "train")
valid_filenames = get_filenames_by_prefix(source_dir, "valid")
test_filenames = get_filenames_by_prefix(source_dir, "test")

import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)

['test_00.csv', 'test_01.csv', 'test_02.csv', 'test_03.csv', 'test_04.csv', 'test_05.csv', 'test_06.csv', 'test_07.csv', 'test_08.csv', 'test_09.csv', 'train_00.csv', 'train_01.csv', 'train_02.csv', 'train_03.csv', 'train_04.csv', 'train_05.csv', 'train_06.csv', 'train_07.csv', 'train_08.csv', 'train_09.csv', 'train_10.csv', 'train_11.csv', 'train_12.csv', 'train_13.csv', 'train_14.csv', 'train_15.csv', 'train_16.csv', 'train_17.csv', 'train_18.csv', 'train_19.csv', 'valid_00.csv', 'valid_01.csv', 'valid_02.csv', 'valid_03.csv', 'valid_04.csv', 'valid_05.csv', 'valid_06.csv', 'valid_07.csv', 'valid_08.csv', 'valid_09.csv']
['./generate_csv/train_00.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_04.csv',
 './generate_csv/train_05.csv',
 './generate_csv/train_06.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_08.csv',
 './generate_csv/train_09.csv',
 './generate_csv/train_10.csv',
 './generate_csv/

In [4]:
# copy读取csv文件的代码，修改
def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(lambda filename: tf.data.TextLineDataset(filename).skip(1),
                                 cycle_length=n_readers)
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset
    
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

batch_size = 32
train_set = csv_reader_dataset(train_filenames, batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames, batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames, batch_size=batch_size)

In [5]:
# 遍历train_set、valid_set、test_set，数据写入tfrecord文件中
# step 1:
def serialize_example(x, y):
    "converts x, y to tf.train.Example and serialize"
    # 这里自己加了.numpy() 不然报错希望real num
    input_features = tf.train.FloatList(value = x.numpy())
    label = tf.train.FloatList(value = y.numpy())
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(float_list = input_features),
            "label": tf.train.Feature(float_list = label)
        }       
    )
    example = tf.train.Example(features = features)
    return example.SerializeToString()

# step 2:
def csv_dataset_to_tfrecords(base_filename, dataset, n_shards,
                            steps_per_shard, compression_type=None):
    options = tf.io.TFRecordOptions(compression_type=compression_type)
    all_filenames = []
    for shard_id in range(n_shards):
        filename_fullpath = '{}_{:05d}-of-{:05d}'.format(
            base_filename, shard_id, n_shards)
        with tf.io.TFRecordWriter(filename_fullpath, options) as writer:
            for x_batch, y_batch in dataset.take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(serialize_example(x_example, y_example))
        all_filenames.append(filename_fullpath)
    return all_filenames

In [18]:
# 调用
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard, None)
test_tfrecord_filenames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard, None)

In [15]:
# 读取文件
expected_features = {
    "input_features" : tf.io.FixedLenFeature([8], dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
}
def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"], example["label"]

def tfrecords_reader_dataset(filenames, n_readers=5, batch_size=32,
                       n_parse_threads=5, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(lambda filename: tf.data.TFRecordDataset(filename).skip(1),
                                          cycle_length=n_readers)
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

tfrecords_train = tfrecoder_reader_dataset(train_tfrecord_filenames, batch_size=3)
for x_batch, y_batch in tfrecords_train.take(2):
    print(x_batch, y_batch)


tf.Tensor(
[[ 0.81150836 -0.04823952  0.5187339  -0.0293864  -0.03406402 -0.05081595
  -0.7157357   0.91627514]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [ 0.04326301 -1.0895426  -0.38878718 -0.10789865 -0.68186635 -0.0723871
  -0.8883662   0.8213992 ]], shape=(3, 8), dtype=float32) tf.Tensor(
[[2.147]
 [2.419]
 [1.426]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 0.15782312  0.4323619   0.3379948  -0.01588031 -0.37338907 -0.05305246
   0.80061346 -1.2359096 ]
 [-1.0591781   1.3935647  -0.02633197 -0.1100676  -0.6138199  -0.09695935
   0.3247131  -0.03747724]
 [ 0.40127665 -0.92934215 -0.0533305  -0.18659453  0.65456617  0.02643447
   0.9312528  -1.4406418 ]], shape=(3, 8), dtype=float32) tf.Tensor(
[[3.169]
 [0.672]
 [2.512]], shape=(3, 1), dtype=float32)


In [23]:
# 生成训练中使用的训练集
batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(valid_tfrecord_filenames, batch_size = batch_size)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1)
])
model.compile(loss='mse', optimizer='sgd')
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
history = model.fit(tfrecords_train_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_data = tfrecords_valid_set,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 1s 4ms/step - loss: 0.7906 - val_loss: 0.5762
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4714 - val_loss: 0.5054
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4266 - val_loss: 0.4591
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4104 - val_loss: 0.4500
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3986 - val_loss: 0.4269
Epoch 6/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3978 - val_loss: 0.4267
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4177 - val_loss: 0.3996
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4534 - val_loss: 0.4683
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3686 - val_loss: 0.4064
Epoch 10/100
348/348 [=========================

In [24]:
model.evaluate(tfrecords_test_set, steps = 5160 // batch_size)

161/161 [==============================] - 0s 882us/step - loss: 0.3721


0.3720605820880173